In [5]:
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import datetime

In [6]:
client_credentials_manager = SpotifyClientCredentials(client_id='ac1f41b0f93c45ac8af29623bdf94e0a', client_secret='f2460ff1a77e4c6db5eca633dde46d3b')
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)


# playlist_link = "https://open.spotify.com/playlist/37i9dQZF1DX0XUfTFmNBRM?si=7934cecad3f3439c"
playlist_link = input("Enter the link: ")

playlist_URI = playlist_link.split("/")[-1].split("?")[0]

songs = sp.playlist_tracks(playlist_URI, market='IN')['items']  #will give a list of songs (meta-data)

Enter the link: https://open.spotify.com/playlist/37i9dQZF1DXdpQPPZq3F7n?si=26c2cf6415a1433c


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [438]:
song_name = []
song_id = []
song_popu = []
song_added_date = []
song_release_date = []
for song in songs:
    song_name.append(song['track']['name'])
    song_id.append(song['track']['id'])
    song_popu.append(song['track']['popularity'])
    song_added_date.append(song['added_at'])
    song_release_date.append(song['track']['album']['release_date'])
 
release_year = map(lambda x: int(x[:4]), song_release_date)

In [439]:
#combining the data
import pandas as pd

df = pd.DataFrame({
    'name':song_name,
    'popularity':song_popu,
    'date_added':song_added_date,
    'release_year':release_year
    })

In [440]:
#audio features

features = sp.audio_features(song_id)

feat_names = list(sp.audio_features(song_id)[0].keys())

for row in range(len(features)):
    for col in range(len(feat_names)):
        df.loc[row, feat_names[col]] = features[row][feat_names[col]]
# df

In [441]:
#artists
artists_col = []
for i in range(len(songs)):
    all_artists = songs[i]['track']['artists']
    artists = []
    for a in all_artists:
        artists.append(a['name'])
    artists_col.append(artists)
df['artists'] = artists_col

In [442]:
df['artists']

0                                                  [KK]
1                          [Pritam, Shaan, Sultan Khan]
2                                    [Sukhwinder Singh]
3                   [Atif Aslam, Alisha Chinai, Pritam]
4                                          [Rashid Ali]
                            ...                        
95    [Shankar Mahadevan, Sunidhi Chauhan, Jaspinder...
96                                   [Mahalakshmi Iyer]
97                                 [Vishal-Shekhar, KK]
98                            [KK, Dj A-Myth, Dj Kiran]
99              [Alka Yagnik, Udit Narayan, Kumar Sanu]
Name: artists, Length: 100, dtype: object

In [443]:
df['popularity'] = list(map(lambda x: x // 5, df['popularity']))

for i in range(0,21):
    df[f"popu|{i}"] = [0] * len(df['name'])

for i in range(1980, datetime.date.today().year + 1):
    df[f"year|{i}"] = [0] * len(df['name'])

df_year = pd.get_dummies(df['release_year'])
df_popu = pd.get_dummies(df['popularity'])
df_year.columns = map(lambda x: 'year' + '|' + str(x), df_year.columns)
df_popu.columns = map(lambda x: 'popu' + '|' + str(x), df_popu.columns)
df_popu.head()

for col in df_popu.columns:
    df[col] = df_popu[col]
for col in df_year.columns:
    df[col] = df_year[col]

In [444]:
new_df = df.copy()
new_df['artists']

0                                                  [KK]
1                          [Pritam, Shaan, Sultan Khan]
2                                    [Sukhwinder Singh]
3                   [Atif Aslam, Alisha Chinai, Pritam]
4                                          [Rashid Ali]
                            ...                        
95    [Shankar Mahadevan, Sunidhi Chauhan, Jaspinder...
96                                   [Mahalakshmi Iyer]
97                                 [Vishal-Shekhar, KK]
98                            [KK, Dj A-Myth, Dj Kiran]
99              [Alka Yagnik, Udit Narayan, Kumar Sanu]
Name: artists, Length: 100, dtype: object

In [445]:
artists_excel = pd.read_excel('datasets/artists_names.xlsx')

excel_df = pd.read_csv("datasets/final_data.csv")

In [446]:
# for converting the stringed list to again list

# print(excel_df.loc[i, 'artists'])
art = []
for i in range(len(excel_df['name'])):
    b = 0
    ele = ''
    new_l = []
    for ch in excel_df.loc[i, 'artists']:
        if b == 1:
            if ch == "'":
                b = 0
                new_l.append(ele)
                ele = ''
            else:
                ele += ch
        elif ch == "'":
            b = 1

    art.append(new_l)
print(art)
# excel_df['artists'] = art
# excel_df['artists']
    # print(new_l)
    # print(excel_df.loc[i, 'artists'])

excel_df['artists'] = art

[['Harrdy Sandhu'], ['Badshah', 'Payal Dev'], ['Badshah', 'Nikhita Gandhi'], ['Badshah'], ['Guru Randhawa', 'Nikhita Gandhi', 'Tanishk Bagchi', 'Nora Fatehi'], ['Sachet Tandon', 'Nikhita Gandhi', 'Madhubanti Bagchi', 'Sachin-Jigar'], ['Jubin Nautiyal', 'Meet Bros.', 'Shardul Rathod'], ['Tanishk Bagchi', 'Jubin Nautiyal', 'Asees Kaur'], ['Pritam', 'Arijit Singh'], ['Vishal-Shekhar', 'Arijit Singh', 'Shilpa Rao', 'Kumaar'], ['Vishal Dadlani', 'Benny Dayal'], ['Pritam', 'Arijit Singh', 'Neha Kakkar', 'Meet Bros.'], ['Sachet Tandon', 'Dhvani Bhanushali', 'Tanishk Bagchi'], ['Hasan Shah'], ['B Praak', 'Harrdy Sandhu', 'Swasti Mehul'], ['A.R. Rahman', 'Shreya Ghoshal', 'Irshad Kamil'], ['A.R. Rahman', 'Shreya Ghoshal'], ['Vishal-Shekhar', 'Arijit Singh', 'Caralisa Monteiro', 'Jaideep Sahni'], ['Yo Yo Honey Singh'], ['Badshah'], ['Badshah', 'Neha Kakkar', 'Varun Dhawan'], ['Badshah', 'Neeti Mohan'], ['Arijit Singh'], ['Palak Muchhal'], ['Vayu', 'Aastha Gill', 'AKASA', 'Puri'], ['Arijit Singh'

In [447]:
zeros = [0] * len(new_df['name'])
extra = pd.DataFrame(zeros)
# for name in artists_excel['artists']:
#     extra[f"artist|{name}"] = 0

for name in artists_excel['artists']:
    extra[f"artist|{name}"] = 0

new_df = pd.concat([new_df, extra], axis=1)
new_df.head()
excel_df = pd.concat([excel_df, new_df])

C:\Users\Shrey\AppData\Local\Temp\ipykernel_6236\4254359574.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  extra[f"artist|{name}"] = 0
C:\Users\Shrey\AppData\Local\Temp\ipykernel_6236\4254359574.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  extra[f"artist|{name}"] = 0
C:\Users\Shrey\AppData\Local\Temp\ipykernel_6236\4254359574.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usin

In [448]:
# pd.concat([excel_df, new_df])

In [449]:
# artists_excel = pd.read_excel('datasets/artists_names.xlsx')

# excel_df = pd.read_excel("datasets/final_data.xlsx")


# zeros = [0] * len(new_df['name'])
# extra = pd.DataFrame(zeros)

# for name in artists_excel['artists']:
#     extra[f"artist|{name}"] = 0

# new_df = pd.concat([new_df, extra], axis=1)
# new_df.head()
# excel_df = pd.concat([excel_df, new_df])

In [450]:
print(len(excel_df), len(df))

595 100


In [451]:
excel_df = pd.concat([excel_df, new_df])
for name in artists_excel['artists']:
    excel_df[f"artist|{name}"] = 0
excel_df.loc[:, 'year|2023':]

,year|2023,year|0,artist|Pritam,artist|Arijit Singh,artist|Amitabh Bhattacharya,artist|Kaifi Khalil,artist|Tanishk Bagchi,artist|Jubin Nautiyal,artist|Asees Kaur,artist|King,...,artist|Miss Congeniality,artist|Torpe,artist|Baby Tyson,artist|Seb Tik,artist|Confetti,artist|Victoria Justice,artist|Tainy,artist|Glee Cast,artist|Jamie Miller,0
0,0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
1,0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
2,0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
3,0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
4,0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,NaN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
96,0,NaN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
97,0,NaN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
98,0,NaN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0


In [452]:
excel_df.drop([0], axis=1, inplace=True)

In [453]:
len(excel_df)

695

In [454]:
# excel_df.head(20)
# ex = excel_df.copy()
# ex.reset_index(inplace=True)
# ex.drop(['index'], axis=1, inplace=True)
# # ex

In [455]:
# ex['artists'] = art
# excel_df = ex.copy()
# excel_df

In [456]:
excel_df = excel_df.reset_index().drop('index', axis=1)
excel_df.iloc[76, :]
# np.array(excel_df.loc[75, 'artists'])

Unnamed: 0                                                     76.0
name                       Galliyan (Unplugged) [From "Ek Villain"]
popularity                                                        9
date_added                                     2022-09-29T21:24:03Z
release_year                                                   2017
                                             ...                   
artist|Confetti                                                   0
artist|Victoria Justice                                           0
artist|Tainy                                                      0
artist|Glee Cast                                                  0
artist|Jamie Miller                                               0
Name: 76, Length: 1320, dtype: object

In [457]:
# excel_df = excel_df.reset_index()
# excel_df.drop('index', axis=1, inplace=True)

for i in range(len(excel_df['name'])):
    # print(excel_df.loc[i, 'artists'])
    for name in np.array(excel_df.loc[i, 'artists']):
        if name in np.array(artists_excel['artists']):
            excel_df.loc[i, f"artist|{name}"] = 1

# for i, row in excel_df.iterrows():
#     for name in row['artists']:
#         print(list(row['artists']))
#         # print(artists_excel['artists'])
#         if name in list(artists_excel['artists']):
#             excel_df.loc[i, f"artist|{name}"] = 1

# excel_df = excel_df.drop(0, axis=1)

In [458]:
list(excel_df.columns)

['Unnamed: 0',
 'name',
 'popularity',
 'date_added',
 'release_year',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'type',
 'id',
 'uri',
 'track_href',
 'analysis_url',
 'duration_ms',
 'time_signature',
 'artists',
 'popu|0',
 'popu|1',
 'popu|2',
 'popu|3',
 'popu|4',
 'popu|5',
 'popu|6',
 'popu|7',
 'popu|8',
 'popu|9',
 'popu|10',
 'popu|11',
 'popu|12',
 'popu|13',
 'popu|14',
 'popu|15',
 'popu|16',
 'popu|17',
 'popu|18',
 'popu|19',
 'popu|20',
 'year|1980',
 'year|1981',
 'year|1982',
 'year|1983',
 'year|1984',
 'year|1985',
 'year|1986',
 'year|1987',
 'year|1988',
 'year|1989',
 'year|1990',
 'year|1991',
 'year|1992',
 'year|1993',
 'year|1994',
 'year|1995',
 'year|1996',
 'year|1997',
 'year|1998',
 'year|1999',
 'year|2000',
 'year|2001',
 'year|2002',
 'year|2003',
 'year|2004',
 'year|2005',
 'year|2006',
 'year|2007',
 'year|2008',
 'year|2009',
 'year|2010',
 'y

In [459]:
# excel_df['name|artists'] = list(map(lambda x, y: f"{x}|{str(y)}", excel_df['name'], excel_df['artists']))

In [460]:
excel_df.columns
excel_df['id']

0      1iZLpuGMr4tn1F5bZu32Kb
1      0eiX2zTStU6Cz4hjUWnUa1
2      2NTVtNcpC0i1R7LVGaCZCF
3      6iSUIEoQaZXB2gtTCTeQYv
4      1orM67pmkNc4alrZkWjYkV
                ...          
690    28syjZyasVJ4J4oxBDOF1b
691    4WPH9jLeFBuMPmYI5NWJgJ
692    2vuhZrEHS25D8xNQTSvuuj
693    11MVHPAE4fu7rbGuNboXqs
694    5yKaiBEY3IrnGUYt3mpyEe
Name: id, Length: 695, dtype: object

In [461]:
excel_df = excel_df.drop_duplicates('id')
excel_df.columns

Index(['Unnamed: 0', 'name', 'popularity', 'date_added', 'release_year',
       'danceability', 'energy', 'key', 'loudness', 'mode',
       ...
       'artist|Lost Frequencies', 'artist|Miss Congeniality', 'artist|Torpe',
       'artist|Baby Tyson', 'artist|Seb Tik', 'artist|Confetti',
       'artist|Victoria Justice', 'artist|Tainy', 'artist|Glee Cast',
       'artist|Jamie Miller'],
      dtype='object', length=1320)

In [462]:
len(excel_df)

591

In [463]:
excel_df = excel_df.reset_index().drop('index', axis=1)
excel_df.columns

Index(['Unnamed: 0', 'name', 'popularity', 'date_added', 'release_year',
       'danceability', 'energy', 'key', 'loudness', 'mode',
       ...
       'artist|Lost Frequencies', 'artist|Miss Congeniality', 'artist|Torpe',
       'artist|Baby Tyson', 'artist|Seb Tik', 'artist|Confetti',
       'artist|Victoria Justice', 'artist|Tainy', 'artist|Glee Cast',
       'artist|Jamie Miller'],
      dtype='object', length=1320)

In [466]:
# excel_df = pd.read_excel("final_data.xlsx")
# pd.concat([excel_df, new_df])
if 'year|0' in excel_df.columns:
    excel_df = excel_df.drop('year|0', axis=1)
if 'Unnamed: 0' in excel_df.columns:
    excel_df = excel_df.drop('Unnamed: 0', axis=1)
if "Unnamed: 0.1" in excel_df.columns:
    excel_df = excel_df.drop('Unnamed: 0.1', axis=1)
if "Unnamed: 0.2" in excel_df.columns:
    excel_df = excel_df.drop('Unnamed: 0.2', axis=1)
excel_df.to_csv('datasets/final_data.csv')

In [467]:
excel_df.to_csv('f.csv')
excel_df['artists']

0                                        [Harrdy Sandhu]
1                                   [Badshah, Payal Dev]
2                              [Badshah, Nikhita Gandhi]
3                                              [Badshah]
4      [Guru Randhawa, Nikhita Gandhi, Tanishk Bagchi...
                             ...                        
586    [Shankar Mahadevan, Sunidhi Chauhan, Jaspinder...
587                                   [Mahalakshmi Iyer]
588                                 [Vishal-Shekhar, KK]
589                            [KK, Dj A-Myth, Dj Kiran]
590              [Alka Yagnik, Udit Narayan, Kumar Sanu]
Name: artists, Length: 591, dtype: object